In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\engineered_data.csv")


C:\Users\harsh\AppData\Local\Temp\ipykernel_14144\1957941901.py:4: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\engineered_data.csv")


In [10]:
# Only keep rows with both msisdn and week_number
filtered_df = df.dropna(subset=['msisdn', 'week_number'])

# Make sure week_number is numeric
filtered_df['week_number'] = pd.to_numeric(filtered_df['week_number'], errors='coerce')
filtered_df = filtered_df.dropna(subset=['week_number'])


C:\Users\harsh\AppData\Local\Temp\ipykernel_14144\3317831702.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['week_number'] = pd.to_numeric(filtered_df['week_number'], errors='coerce')


In [11]:
kpi_df = filtered_df.groupby('msisdn').agg(
    total_revenue=('revenue_usd', 'sum'),
    active_weeks=('week_number', pd.Series.nunique)
).reset_index()

# Avoid divide-by-zero
kpi_df['avg_revenue_per_week'] = kpi_df['total_revenue'] / kpi_df['active_weeks']
kpi_df = kpi_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['avg_revenue_per_week'])


In [12]:
kpi_df.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\kpi_by_user.csv", index=False)


In [13]:
# active users with zero revenue
kpi_df['is_zero_revenue_but_active'] = (kpi_df['total_revenue']==0)&(kpi_df['active_weeks']>0)
print(kpi_df['is_zero_revenue_but_active'].head())

0    False
1    False
2    False
3    False
4    False
Name: is_zero_revenue_but_active, dtype: bool


In [14]:
# get threshold for high activity and low revenue for high usage and low revenue kpi
high_activity_threshold = kpi_df['active_weeks'].quantile(0.75)
low_revenue_threshold = kpi_df['avg_revenue_per_week'].quantile(0.25)

# high usage but low revenue users
kpi_df['is_high_usage_low_revenue']=(
    (kpi_df['active_weeks'] >=high_activity_threshold) &
    (kpi_df['avg_revenue_per_week'] <=low_revenue_threshold)
)

In [15]:
kpi_df.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\kpi_by_user.csv")

In [16]:
kpi_df.drop(columns=['is_zero_revenue_but-active'],inplace=True)

KeyError: "['is_zero_revenue_but-active'] not found in axis"

In [17]:
kpi_df.columns.tolist()

['msisdn',
 'total_revenue',
 'active_weeks',
 'avg_revenue_per_week',
 'is_zero_revenue_but_active',
 'is_high_usage_low_revenue']

In [18]:
kpi_df.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\kpi_by_user.csv")

In [17]:
# Revenue segmentation 

In [19]:
kpi_df['revenue_segment'] = pd.qcut(
    kpi_df['avg_revenue_per_week'],
    q=4,
    labels=['Low','Medium','High','Very High']
)


In [20]:
segment_count = kpi_df['revenue_segment'].value_counts()
print(segment_count)

revenue_segment
Low          24338
Medium       24338
Very High    24338
High         24337
Name: count, dtype: int64


In [21]:
print(kpi_df['is_zero_revenue_but_active'].value_counts())

is_zero_revenue_but_active
False    97351
Name: count, dtype: int64


In [22]:
print(kpi_df['is_high_usage_low_revenue'].value_counts())

is_high_usage_low_revenue
False    83749
True     13602
Name: count, dtype: int64


In [23]:
engineer_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\engineered_data.csv")
engineer_df.columns.tolist()

C:\Users\harsh\AppData\Local\Temp\ipykernel_14144\3864983619.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  engineer_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\engineered_data.csv")


['msisdn',
 'gender',
 'year_of_birth',
 'system_status',
 'mobile_type',
 'value_segment',
 'imei_tac',
 'brand_name',
 'model_name',
 'os_name',
 'os_vendor',
 'week_number',
 'revenue_usd',
 'age',
 'device_segment',
 'avg_weekly_revenue',
 'is_gender_missing',
 'revenue_category']

In [25]:
# combine leakage indicator
kpi_df['is_revenue_leakage']= (
    (kpi_df['is_zero_revenue_but_active']) |
    (kpi_df['is_high_usage_low_revenue'])
)

In [ ]:
# count how many users flagged
leakage_count = kpi_df['is_revenue_leakage'].sum()
print(f"Potential Revenue Leakage Users : {leakage_count}")

leakage_users = kpi_df[kpi_df['is_revenue_leakage']]
leakage_users.head()


Potential Revenue Leakage Users : 13602


,msisdn,total_revenue,active_weeks,avg_revenue_per_week,is_zero_revenue_but_active,is_high_usage_low_revenue,revenue_segment,is_revenue_leakage
5,0002b98e027e03cc6ffc266c18f8e6bc,146.887161,14,10.491940,False,True,Low,True
20,001299709d14b07c5ed55d497fe7e6cf,147.212282,14,10.515163,False,True,Low,True
26,0015ba2901be893fbb88a7bad0c38fcb,129.039265,14,9.217090,False,True,Low,True
33,001bb38768d4b469cf516acfd4bbe796,135.171542,14,9.655110,False,True,Low,True
53,002681d3e4ee62ac6dec560370561e59,138.447116,14,9.889080,False,True,Low,True


In [26]:
engineer_df = engineer_df.drop_duplicates(subset='msisdn')


# Merge on msisdn
final_leakage_data = pd.merge(
    engineer_df,
    kpi_df[['msisdn','is_revenue_leakage']],
    on='msisdn',
    how='left'
)

# fill missing leakage flags with false (non_leakage users)
final_leakage_data['is_revenue_leakage'] = final_leakage_data['is_revenue_leakage'].fillna(False)



C:\Users\harsh\AppData\Local\Temp\ipykernel_14144\4241174783.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_leakage_data['is_revenue_leakage'] = final_leakage_data['is_revenue_leakage'].fillna(False)


In [3]:
final_leakage_data.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\final_leakage_data.csv")

NameError: name 'final_leakage_data' is not defined

In [27]:
final_leakage_data.columns.tolist()

['msisdn',
 'gender',
 'year_of_birth',
 'system_status',
 'mobile_type',
 'value_segment',
 'imei_tac',
 'brand_name',
 'model_name',
 'os_name',
 'os_vendor',
 'week_number',
 'revenue_usd',
 'age',
 'device_segment',
 'avg_weekly_revenue',
 'is_gender_missing',
 'revenue_category',
 'is_revenue_leakage']

In [28]:
final_leakage_data['is_revenue_leakage'].value_counts()

is_revenue_leakage
False    11235568
True        13602
Name: count, dtype: int64

In [39]:
kpi_df[['total_revenue','active_weeks']].dtypes

total_revenue    float64
active_weeks       int64
dtype: object

In [40]:
kpi_df['is_revenue_leakage'].value_counts()

is_revenue_leakage
False    83749
True     13602
Name: count, dtype: int64

In [43]:
print(engineer_df['msisdn'].is_unique)
print(kpi_df['msisdn'].is_unique)

False
True


In [2]:
# for visualization we will drop some unnecessary columns from daata
final_leakage_data.isnull.sum()

NameError: name 'final_leakage_data' is not defined

In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\final_leakage_data.csv")

df['week_number'].value_counts()
df['revenue_usd'].value_counts()
df['avg_weekly_revenue'].value_counts()

C:\Users\harsh\AppData\Local\Temp\ipykernel_11252\1086424864.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\final_leakage_data.csv")


avg_weekly_revenue
14.804682    1
10.805290    1
14.746683    1
9.618943     1
12.121832    1
            ..
12.321476    1
8.680809     1
10.114989    1
9.348162     1
11.814107    1
Name: count, Length: 97351, dtype: int64

In [4]:
total_rows = len(df)
print("total number of rows are",total_rows)

print(f"null week number are {df['week_number'].isnull().sum()}")
print(f"Rows week contains actual number {df['week_number'].notnull().sum()}")

total number of rows are 11249170
null week number are 11151819
Rows week contains actual number 97351


In [7]:
def counts(col_name):
    total_rows = len(df)
    print(f"total number of rows are ",total_rows)
    print(f"Total number of rows having null values for column{col_name} are {df[col_name].isnull().sum()}")
    print(f"Total number of rows with actual values for column{col_name} are {df[col_name].notnull().sum()}")

col_name = ['week_number','revenue_usd','avg_weekly_revenue']

for col in col_name:
    counts(col)

total number of rows are  11249170
Total number of rows having null values for columnweek_number are 11151819
Total number of rows with actual values for columnweek_number are 97351
total number of rows are  11249170
Total number of rows having null values for columnrevenue_usd are 11151819
Total number of rows with actual values for columnrevenue_usd are 97351
total number of rows are  11249170
Total number of rows having null values for columnavg_weekly_revenue are 11151819
Total number of rows with actual values for columnavg_weekly_revenue are 97351


In [8]:
customer_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\customer_df.csv")
device_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\device_data.csv")
revenue_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\Revenue_data.csv")
kpi_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\kpi_by_user.csv")

print("Customer:", len(customer_df))
print("Revenue:", len(revenue_df))
print("Device:", len(device_df))
print("KPI by user:", len(kpi_df))



Customer: 13620180
Revenue: 1704601
Device: 2409654
KPI by user: 97351


In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\final_leakage_data.csv")

C:\Users\harsh\AppData\Local\Temp\ipykernel_9856\115247913.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\final_leakage_data.csv")


In [3]:
df['gender'] = df['gender'].astype(str).str.strip().str.lower()
print(df['gender'].value_counts())

gender
male             8107375
unknown          1803692
female           1138580
not available     193089
m                   5672
u                    319
f                    163
other                 95
.                     51
3                     29
female.               16
]                     11
`                      9
female]                7
+                      6
fe                     4
~                      4
female`                3
b                      3
p                      2
femal                  2
1                      2
female3                2
s                      2
a                      2
0                      2
\\                     2
d                      2
female..               1
femele                 1
0.0-                   1
qqqq                   1
!                      1
male.                  1
q                      1
femal]e                1
df                     1
``                     1
mafele                 1
ma                

In [4]:
# Define known values
male_vals = ['male', 'm', 'male.', 'mal', 'ma', 'mfeale', 'mafele', 'man']
female_vals = ['female', 'f', 'female.', 'fe', 'femal', 'femal.', 'femele', 'femal]e',
               'femalef', 'female3', 'female]', 'female`', 'female..', 'femaleh']
unknown_vals = ['unknown', 'not available', 'other', 'u', '.', ']', '`', '+', '~', 'b', 
                'p', '1', '3', '0', '0.0-', '\\\\', 'a', 'd', '!', 'q', 'qqqq', 'df', 
                '``', 'o', '```', '21', 'pr', 'my', '6', 's', 'swdwdxw']

def standardize_gender(g):
    if g in male_vals:
        return 'Male'
    elif g in female_vals:
        return 'Female'
    elif g in unknown_vals:
        return 'Unknown'
    else:
        return 'Unknown'
    
df['gender'] = df['gender'].apply(standardize_gender)

print(df['gender'].value_counts())

gender
Male       8113052
Unknown    1997336
Female     1138782
Name: count, dtype: int64


In [5]:
df.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\final_leakage_data.csv")